In [38]:
import requests
import datetime

In [2]:
#  Authentication is enabled and cluster connections are encrypted.

# ℹ️  Password for the elastic user (reset with `bin/elasticsearch-reset-password -u elastic`):
#   UhJ=sDusoDu=8a*JJ-H6

# ℹ️  HTTP CA certificate SHA-256 fingerprint:
#   3e100f2162f9c96b65a6c2ad7494b3646a476ea418b7f583d3e107692af47f9d

# ℹ️  Configure Kibana to use this cluster:
# • Run Kibana and click the configuration link in the terminal when Kibana starts.
# • Copy the following enrollment token and paste it into Kibana in your browser (valid for the next 30 minutes):
#   eyJ2ZXIiOiI4LjguMSIsImFkciI6WyIxNzIuMTguMC4yOjkyMDAiXSwiZmdyIjoiM2UxMDBmMjE2MmY5Yzk2YjY1YTZjMmFkNzQ5NGIzNjQ2YTQ3NmVhNDE4YjdmNTgzZDNlMTA3NjkyYWY0N2Y5ZCIsImtleSI6IjRES0JyNGdCWV94OU1qaXBaWkNHOlE5OTBwczVWVHItdUZMR3htdDFIRlEifQ==

# ℹ️ Configure other nodes to join this cluster:
# • Copy the following enrollment token and start new Elasticsearch nodes with `bin/elasticsearch --enrollment-token <token>` (valid for the next 30 minutes):
#   eyJ2ZXIiOiI4LjguMSIsImFkciI6WyIxNzIuMTguMC4yOjkyMDAiXSwiZmdyIjoiM2UxMDBmMjE2MmY5Yzk2YjY1YTZjMmFkNzQ5NGIzNjQ2YTQ3NmVhNDE4YjdmNTgzZDNlMTA3NjkyYWY0N2Y5ZCIsImtleSI6IjRqS0JyNGdCWV94OU1qaXBaWkNKOkwyRGZ5RnM4UVNTbGZLWVp1NVI0N1EifQ==


## new elasticsearch

In [50]:
from elasticsearch import Elasticsearch, helpers



es = Elasticsearch("http://localhost:9400",basic_auth=("elastic","secret"),verify_certs=False)
es.indices.get_alias(index="*")

ObjectApiResponse({'data_description': {'aliases': {}}})

In [101]:
indices=dict(es.indices.get_alias(index="*")).keys()
for index in indices:
    es.options(ignore_status=[400,404]).indices.delete(index=index)

In [ ]:
demographics={}
print(es.search(index="baseline", query={'match' : {"userid":"user"}})["hits"]["hits"][0]["_source"])
res=es.search(index="baseline", query={'match' : {"userid":"user"}})["hits"]["hits"]["_source"]["questionnaire"]

demographics["age"]=res["Dem_age"]
demographics["gender"]=res["Dem_gender"]
demographics["height"]=res["Dem_height"]
demographics["weight"]=res["Dem_weight"]
demographics

In [3]:
es.indices.exists(index="appsettings")

HeadApiResponse(True)

In [80]:
es.options(ignore_status=[400,404]).indices.delete(index='education')

ObjectApiResponse({'acknowledged': True})

In [7]:
def get_user(username):
    res = es.search(index="account", query={'match' : {"_id":username}})
    if not res["hits"]["hits"]:
        return False
    return res.body["hits"]["hits"][0]["_source"]

get_user("stuartadmin")

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'userid': 'stuartgo',
 'password': '$2b$12$phrVybcm8uyeTl9D/cdYoeZxiOHsjddjitMoYWv8lMVu9bMuY1L2a',
 'country': 'nl',
 'clinician': 'NTNU',
 'rights': ['ROLE_ADMIN', 'ACTUATOR'],
 'isaccountnonexpired': True,
 'isaccountnonlocked': True,
 'iscredentialsnonexpired': True,
 'isenabled': True}

In [4]:
es.update(index="appsettings",id="user",doc={"hideOnboarding":True})

ObjectApiResponse({'_index': 'appsettings', '_id': 'user', '_version': 232, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 231, '_primary_term': 6})

In [26]:
res = es.search(index="data_description",query={"match":{"description_type":"exercise"}},size=900)
set(map(lambda x: x["_source"]["level"] ,res.body["hits"]["hits"]))

{1, 2, 3, 4, 5, 6}

In [46]:
es.get(index="education", id="stuartStay active_2").body["_source"]["is_correct"]

False

In [38]:
es.search(index="education",query={"bool":
                                          {"must":[
                                              {"match":{"userid":"stuart"}},
                                                {"match":{"educationid":"Stay active_2"}}
                                          ]
                                          }}
,size=900).body["hits"]["hits"]#[0]["_source"]

[{'_index': 'education',
  '_id': 'stuartStay active_2',
  '_score': 0.8630463,
  '_source': {'educationid': 'Stay active_2',
   'is_quiz': False,
   'is_correct': False,
   'userid': 'stuart',
   'date': 1725367768}}]

In [78]:
doc=es.get(index="data_description", id="Relaxation_1").body["_source"]
doc["longmessage"]="sloopydoopydoo"
# doc["question"]="This is a test question and is not a real question, or is it? How is your back doing, is it better?"
# doc["answer"]=True
es.update(index="data_description",id="Relaxation_1",doc=doc)

ObjectApiResponse({'_index': 'data_description', '_id': 'Relaxation_1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 369, '_primary_term': 1})

In [88]:
doc=es.get(index="plan", id="2NLvvJEBWEmV8-ty19WV").body["_source"]
doc["plan"]["educations"][0]["is_quiz"]=True
doc["plan"]["educations"]
es.update(index="plan", id="2NLvvJEBWEmV8-ty19WV",doc=doc)

ObjectApiResponse({'_index': 'plan', '_id': '2NLvvJEBWEmV8-ty19WV', '_version': 4, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1})

In [20]:
original_exercise = es.search(index="data_description",query={"match":{"exerciseid":"ab_04_03"}},size=900)
original_exercise=original_exercise["hits"]["hits"][0]
original_level = original_exercise["_source"]["level"]
original_type = original_exercise["_source"]["type"]
es.search(index="data_description",query={"bool":
                                          {"must":[
                                              {"match":{"type":original_type}},
                                                {"match":{"level":original_level}}
                                          ],
                                          "must_not":[{"match":{"exerciseid":"ab_04_03"}}]
                                          
                                          }}
,size=900).body["hits"]["hits"][0]["_source"]


{'exerciseid': 'ab_04_01',
 'title': 'Liggende bekkenrotasjon',
 'type': 'Ab',
 'purpose': 'Denne øvelsen styrker magemusklene som i kombinasjon med ryggmusklene stabiliserer ryggen når man beveger seg, står oppreist eller løfter.',
 'instruction': 'Ligg på ryggen med hofte og knær i 90 grader og armene litt ut fra kroppen. Aktiver musklene i mage og korsrygg og roter nedre del av kroppen ved å bevege knærne fra side til side i en kontrollert bevegelse.  ',
 'level': 4,
 'repetitions': 10,
 'sets': 3,
 'time': 5,
 'condition': 'LBP_NP',
 'language': 'nb',
 'link': '',
 'description_type': 'exercise'}

In [106]:
res = es.search(index="achievements", query={'match_all' : {}},size=10000)

doc=res.body["hits"]["hits"]
doc
# es.update(index="plan",id=doc["_id"],doc={"end":datetime.datetime.isoformat(datetime.datetime.now())})

[{'_index': 'achievements',
  '_id': 'thea_SetSleepTool',
  '_score': 1.0,
  '_source': {'userid': 'thea',
   'achievementid': 'SetSleepTool',
   'progress': 0,
   'goal': 1,
   'achievedat': -1}},
 {'_index': 'achievements',
  '_id': 'thea_UseMindfulness',
  '_score': 1.0,
  '_source': {'userid': 'thea',
   'achievementid': 'UseMindfulness',
   'progress': 0,
   'goal': 1,
   'achievedat': -1}},
 {'_index': 'achievements',
  '_id': 'thea_QACompleted',
  '_score': 1.0,
  '_source': {'userid': 'thea',
   'achievementid': 'QACompleted',
   'progress': 0,
   'goal': 1,
   'achievedat': -1}},
 {'_index': 'achievements',
  '_id': 'thea_EducationalCorrectQuiz1',
  '_score': 1.0,
  '_source': {'userid': 'thea',
   'achievementid': 'EducationalCorrectQuiz1',
   'progress': 0,
   'goal': 1,
   'achievedat': -1}},
 {'_index': 'achievements',
  '_id': 'thea_EducationalCorrectQuiz3',
  '_score': 1.0,
  '_source': {'userid': 'thea',
   'achievementid': 'EducationalCorrectQuiz3',
   'progress': 0,
 

In [103]:
res = es.search(index="exercise", query={'match_all' : {}},size=10000)

res.body#["hits"]["hits"][0]["_source"]["plan"]["educations"]
# list(map(lambda x: x["_id"],res["hits"]["hits"]))

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'exercise',
    '_id': 'thea_ab_04_03',
    '_score': 1.0,
    '_source': {'exerciseid': 'ab_04_03',
     'status': 'completed',
     'performed': 1725460069601,
     'repsperformed1': 7,
     'repsperformed2': 3,
     'repsperformed3': 4,
     'userid': 'thea',
     'date': 1725460069}}]}}

In [100]:
es.search(index="achievements", 
          query={"bool":{"must":[
            {'match' : 
                 {"userid":"carmela"}},
            {'match' : 
                 {"achievementid":"Relaxation_1"}}
          ]}},size=10000)

ObjectApiResponse({'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [98]:
num_correct=es.search(index="education",query={"bool":
                                          {"must":[
                                              {"match":{"is_correct":True}},
                                                {"match":{"userid":"carmela"}}
                                          ]
                                         
                                          }}
,size=900).body["hits"]["hits"]

for i in [1,3,5,14,25]:
    

[{'_index': 'education',
  '_id': 'carmelaRelaxation_1',
  '_score': 0.5753642,
  '_source': {'educationid': 'Relaxation_1',
   'is_quiz': True,
   'is_correct': True,
   'userid': 'carmela',
   'date': 1725454879}}]

## Old Elasticsearch

In [1]:
from elasticsearch5 import Elasticsearch




es = Elasticsearch("http://localhost:9200")
es.indices.get_alias(index="*")

GET http://localhost:9200/*/_alias [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/elasticsearch5/connection/http_urllib3.py", line 135, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 798, 

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f912c113c10>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f912c113c10>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
language="en"
query={"query":{'match' : {"_type":"education_"+language}}}
query={"query":{'match_all' : {}}}
res = es.search(index="data_description", body=query,size=10000)
print(len(res["hits"]["hits"]))
res["hits"]["hits"]

In [ ]:
#AXK5QIDn3yIeDN3WNAs4

In [ ]:
res = es.search(index="data_description", body={"query":{'match' : {"_type":"exercise_en"}}},size=100)
list(map(lambda x: x["_source"],res["hits"]["hits"]))

In [7]:
es.search(index="data_description", query={'match' : {"exerciseid":"back_01_01"}},size=100)

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 4.189655, 'hits': [{'_index': 'data_description', '_id': 'TTy6LI4Bhva5Q5uik2Ov', '_score': 4.189655, '_source': {'exerciseid': 'back_01_01', 'title': 'Ettbeinsløft', 'type': 'Back', 'group': '', 'explanation': 'Denne øvelsen vil styrke ryggmusklene dine', 'instruction': 'Ligg på magen med armene plassert under pannen for å sikre at ryggen er i en nøytral stilling. Strekk og løft det ene beinet litt opp fra gulvet og senk deretter beinet rolig ned. Gjenta med det andre beinet. ', 'level': 1.0, 'reps': 10.0, 'sets': 3.0, 'time': 5.0, 'condition': 'LBP', 'old id': 'back_01_01', 'description_type': 'exercise'}}]}})

In [6]:
res = es.search(index="data_description", body={"query":{'match_all' : {}}},size=100)
res

/tmp/ipykernel_2489124/3496215199.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="data_description", body={"query":{'match_all' : {}}},size=100)


ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 354, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'data_description', '_id': 'TTy6LI4Bhva5Q5uik2Ov', '_score': 1.0, '_source': {'exerciseid': 'back_01_01', 'title': 'Ettbeinsløft', 'type': 'Back', 'group': '', 'explanation': 'Denne øvelsen vil styrke ryggmusklene dine', 'instruction': 'Ligg på magen med armene plassert under pannen for å sikre at ryggen er i en nøytral stilling. Strekk og løft det ene beinet litt opp fra gulvet og senk deretter beinet rolig ned. Gjenta med det andre beinet. ', 'level': 1.0, 'reps': 10.0, 'sets': 3.0, 'time': 5.0, 'condition': 'LBP', 'old id': 'back_01_01', 'description_type': 'exercise'}}, {'_index': 'data_description', '_id': 'Tjy6LI4Bhva5Q5uik2Ov', '_score': 1.0, '_source': {'exerciseid': 'backneck_01_01', 'title': 'Enarmsløft', 'type': 'Back', 'group': '', 'explanation': 'Denne øvelsen vil sty

In [ ]:
res = es.search(index="account", body={"query":{'match' : {"userid":"stuart126"}}},size=100)
res

In [4]:
response=requests.post(
    "https://back-up.idi.ntnu.no/oauth/token",
    data={'grant_type': 'password',"username":"stuartFinal2","password":"cWbWaN6qgAVjhxdqqx5Ht"},
    verify=False,
    allow_redirects=False,
    auth=("my-trusted-client","secret")
)
token=response.json()["access_token"]
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'access_token': '9gQtMI9eHG8MdzUMtZMkWf42ymI',
 'token_type': 'bearer',
 'scope': 'read write trust'}

In [6]:
response=requests.get("https://back-up.idi.ntnu.no/data/achievement/list",verify=False,
                      headers={"Authorization": f"Bearer {token}"}
                      )
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'achievementid': 'SetSleepTool',
  'type': 'general',
  'title': 'Sleep Reminder',
  'description': 'Set a sleep reminder using the Sleep Reminder Tool.',
  'description_achieved': 'Set a sleep reminder using the Sleep Reminder Tool.',
  'image': 'https://dvlxmvtyf5r6m.cloudfront.net/achievements/sleep-reminder.png',
  'language': 'en',
  'goal': 1,
  'goaltext': 'Incomplete.'},
 {'achievementid': 'EducationalMaterialRead1',
  'type': 'educational',
  'title': 'Educational Message Read',
  'description': 'Read an educational message.',
  'description_achieved': 'Read an educational message.',
  'image': 'https://dvlxmvtyf5r6m.cloudfront.net/achievements/educational-read-1.png',
  'language': 'en',
  'goal': 1,
  'goaltext': 'Complete.'},
 {'achievementid': 'EducationalMaterialRead25',
  'type': 'educational',
  'title': 'Educational Message Read',
  'description': 'Read 25 educational messages.',
  'description_achieved': 'Read 25 educational messages.',
  'image': 'https://dvlxmvtyf

In [10]:
response=requests.get("https://back-up.idi.ntnu.no/patient/language",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      )
print(response)
response.json()

<Response [200]>


/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'message': 'en'}

In [55]:
response=requests.post("https://back-up.idi.ntnu.no/patient/plan/next",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      json={
                            "exercises_duration": 35,
                            }
                      )
print(response)
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


{'created': 1687337957813,
 'start': 1687305600000,
 'end': 1687910399999,
 'exercises_duration': 35,
 'history': [{'date': 1687337957813,
   'exercises': [{'exerciseid': 'core_01_02',
     'sets': 3,
     'reps': 10,
     'set_duration_s': -1},
    {'exerciseid': 'flex_01_09', 'sets': 1, 'reps': 1, 'set_duration_s': 30},
    {'exerciseid': 'backneck_01_01',
     'sets': 3,
     'reps': 10,
     'set_duration_s': -1},
    {'exerciseid': 'glut_01_02', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'flex_01_03', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'ab_01_02', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'flex_01_08', 'sets': 1, 'reps': 1, 'set_duration_s': 30}],
   'educations': [{'educationid': 'Family and friends_3',
     'is_quiz': False,
     'is_correct': False},
    {'educationid': 'Problem solving_3',
     'is_quiz': False,
     'is_correct': False},
    {'educationid': 'Stay active_10', 'is_quiz': False, 'is_corre

In [11]:
response=requests.post("http://localhost:30000/token",data={"username":"stuartgo","password":"secret"}
)
token=response.json()["access_token"]
response.json()

NameError: name 'requests' is not defined

In [58]:
response=requests.post("https://localhost:30000/patient/plan/next",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      json={
                            "exercises_duration": 35,
                            }
                      )
print(response)
response.json()

SSLError: HTTPSConnectionPool(host='localhost', port=30000): Max retries exceeded with url: /patient/plan/next (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))

In [20]:
response=requests.get("http://localhost:30000/data/patient/baseline",headers={"Authorization": f"Bearer {token}"}
)
response.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=30000): Max retries exceeded with url: /data/patient/baseline (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f34d6b8e070>: Failed to establish a new connection: [Errno 111] Connection refused'))